In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-code/Customer_Code1.csv
/kaggle/input/cleared-dataset/Nan_Filled.csv
/kaggle/input/12345678/train_action_history.csv
/kaggle/input/12345678/train_cdna_data.csv


In [3]:

df = pd.read_csv('/kaggle/input/cleared-dataset/Nan_Filled.csv')

In [4]:
df.shape

(1285402, 124)

In [6]:
df.head()

,v2,v5,v7,v24,v30,v32,v42,v44,v45,v51,...,v293,v294,v295,v296,v297,v298,v299,v300,v301,v302
0,50.0,99.0,99999.0,0.0,11.0,50.0,0,4.0,1,1,...,306,187,44,204,150,73,271,164,170,246
1,35.0,99.0,95013.0,0.0,15.0,60.0,0,4.0,1,0,...,306,187,44,204,150,73,271,164,170,246
2,20.0,99.0,95013.0,0.0,95.0,50.0,0,4.0,1,1,...,306,187,44,204,150,73,271,164,170,246
3,20.0,99.0,95013.0,0.0,1.0,84.0,0,4.0,1,0,...,306,187,44,204,150,73,271,164,170,246
4,30.0,99.0,95013.0,0.0,4.0,60.0,0,4.0,1,0,...,306,187,44,204,150,73,271,164,170,246


In [7]:
pd.set_option('display.max_columns', None)

In [8]:
df2=pd.read_csv('/kaggle/input/customer-code/Customer_Code1.csv')

In [9]:
df2.shape

(1285402, 2)

In [10]:
df2.head()

,CUSTOMER_CODE,batch_date
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,2024-09-27
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,2024-09-27
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,2024-09-27
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,2024-09-27
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,2024-09-27


In [11]:
from sklearn.feature_selection import VarianceThreshold

# Define a variance threshold (e.g., 0.01 for very low variance)
threshold = 0.01

# Apply VarianceThreshold
selector = VarianceThreshold(threshold=threshold)
df_reduced = df.loc[:, selector.fit(df).get_support()]

# Display the result
print(f"Columns removed: {list(set(df.columns) - set(df_reduced.columns))}")

Columns removed: ['v142', 'v147', 'v274', 'v141', 'v42', 'v68', 'v140', 'v148']


In [12]:
df_reduced.shape

(1285402, 116)

In [13]:
correlation_threshold = 0.9

# Calculate the correlation matrix
correlation_matrix = df_reduced.corr()

# Find columns to drop
# Select the upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

# Find columns with correlation above the threshold
to_drop = [
    column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)
]

# Drop the highly correlated columns
df_reduced = df_reduced.drop(columns=to_drop)


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


In [14]:
df_reduced.shape

(1285402, 79)

In [ ]:
df_reduced.describe()

In [15]:
binary_columns = df_reduced.select_dtypes(include=['int', 'float']).columns
continuous_columns = df_reduced.select_dtypes(exclude=['int', 'float']).columns

In [21]:
pd.set_option('display.max_rows', None)

In [22]:
unique_values_count = df_reduced.nunique()
unique_values_count

v2          22
v5          22
v7         197
v24          2
v30         63
v32        208
v44          4
v45          2
v51          2
v52          2
v53          2
v54          2
v63          2
v77         23
v81          2
v82          2
v87          2
v88          2
v92          2
v95          2
v99          2
v103         2
v104         2
v105         2
v106         2
v110        40
v116       193
v128      2490
v172        10
v173         3
v174        30
v175         6
v176        65
v177        12
v178       126
v203      3194
v204      3156
v206        84
v207    415596
v208        50
v209        68
v210      4404
v211      4233
v213        86
v215        50
v216        49
v224      4498
v225      4179
v227       113
v238      5806
v239      5601
v241       321
v243        42
v244        43
v255       357
v257       100
v258       102
v259       413
v263       121
v271         2
v273         2
v275         2
v276         2
v277         2
v278         2
v279         2
v280      

In [24]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# List of columns to apply PCA on
pca_columns = ['v2', 'v5', 'v7', 'v30', 'v32', 'v77', 'v110', 'v116', 'v128', 
               'v172', 'v174', 'v176', 'v177', 'v178', 'v203', 'v204', 'v206', 
               'v207', 'v208', 'v209', 'v210', 'v211', 'v213', 'v215', 'v216', 
               'v224', 'v225', 'v227', 'v238', 'v239', 'v241', 'v243', 'v244', 
               'v255', 'v257', 'v258', 'v259', 'v263']

# Selecting the specified columns from the DataFrame for PCA
df_selected = df_reduced[pca_columns]

# Standardizing the selected data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_selected)

# Applying PCA
pca = PCA()
pca_data = pca.fit_transform(scaled_data)

# Calculating the explained variance ratio
explained_variance = pca.explained_variance_ratio_

# Finding the number of components that capture 95% of the variance
cumulative_variance = explained_variance.cumsum()
num_components = (cumulative_variance <= 0.95).sum() + 1

# Applying PCA with the selected number of components
pca_final = PCA(n_components=num_components)
pca_reduced_data = pca_final.fit_transform(scaled_data)

# Creating a DataFrame for the reduced data
pca_columns = [f'PC{i+1}' for i in range(num_components)]
pca_df = pd.DataFrame(pca_reduced_data, columns=pca_columns)

# Recombining the PCA-reduced data with the original DataFrame (excluding PCA columns)
# First drop only the columns used in PCA transformation
df_non_pca = df_reduced.drop(columns=pca_columns, errors='ignore')

# Concatenate the reduced PCA data with the original (non-PCA) columns
df_final = pd.concat([df_non_pca, pca_df], axis=1)

# Show the final DataFrame with PCA applied
# print(df_final)


In [26]:
# List of columns to drop
columns_to_drop = ['v2', 'v5', 'v7', 'v30', 'v32', 'v77', 'v110', 'v116', 'v128', 
               'v172', 'v174', 'v176', 'v177', 'v178', 'v203', 'v204', 'v206', 
               'v207', 'v208', 'v209', 'v210', 'v211', 'v213', 'v215', 'v216', 
               'v224', 'v225', 'v227', 'v238', 'v239', 'v241', 'v243', 'v244', 
               'v255', 'v257', 'v258', 'v259', 'v263']

# Drop the specified columns
df_final = df_final.drop(columns=columns_to_drop, errors='ignore')


In [25]:
df_final.head(10)

,v2,v5,v7,v24,v30,v32,v44,v45,v51,v52,v53,v54,v63,v77,v81,v82,v87,v88,v92,v95,v99,v103,v104,v105,v106,v110,v116,v128,v172,v173,v174,v175,v176,v177,v178,v203,v204,v206,v207,v208,v209,v210,v211,v213,v215,v216,v224,v225,v227,v238,v239,v241,v243,v244,v255,v257,v258,v259,v263,v271,v273,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v290,v291,v295,v298,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26
0,50.0,99.0,99999.0,0.0,11.0,50.0,4.0,1,1,1,0,0,0,101.0,0.0,0,1,1,1,1,0,1,0,0,0,28.0,28.0,28.181818,41.0,2.0,75.0,4.0,164.0,6.0,348.0,572.0,479.0,1.0,282011.00,22101.0,22101.0,674.0,448.0,1.0,43552.0,43552.0,668.0,411.0,1.0,2234.0,1655.0,2.0,43554.0,43301.0,2.0,43554.0,43301.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,103,278,362,21,44,73,-0.322119,-0.918599,-0.273532,-0.559537,-0.386782,-0.089112,1.685201,-0.336442,-1.153682,-0.526113,-0.063434,0.314679,0.202600,0.345686,-0.211092,-0.154966,0.062258,-0.355990,-0.011503,-0.630801,0.055311,-0.097643,0.252998,0.167742,-0.113037,-0.181209
1,35.0,99.0,95013.0,0.0,15.0,60.0,4.0,1,0,0,1,0,1,101.0,0.0,1,0,0,0,0,1,1,1,0,1,28.0,28.0,28.181818,41.0,2.0,75.0,4.0,164.0,6.0,348.0,2248.0,418.0,3.0,1008460.78,22101.0,23201.0,674.0,448.0,1.0,43552.0,43552.0,2248.0,418.0,3.0,1254.0,918.0,1.0,43552.0,43552.0,1.0,43551.0,43551.0,6.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,103,278,362,21,44,73,2.553447,0.951081,0.300505,0.231803,-0.014947,-0.501481,-0.235533,0.353483,-0.696045,0.090170,0.981325,1.183267,0.818103,-0.558489,-0.615532,-0.256515,-0.087198,0.350292,-0.971604,-0.149342,0.140371,0.823861,0.101140,0.074615,-0.499478,-0.387337
2,20.0,99.0,95013.0,0.0,95.0,50.0,4.0,1,1,1,0,0,0,101.0,1.0,0,0,1,0,0,0,1,0,1,0,28.0,28.0,28.181818,41.0,2.0,75.0,4.0,164.0,6.0,348.0,572.0,479.0,1.0,282011.00,22101.0,22101.0,1086.0,1086.0,1.0,43552.0,43552.0,1086.0,1086.0,1.0,1254.0,918.0,1.0,43552.0,43552.0,1.0,43551.0,43551.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,1.056558,-0.682596,-0.784958,0.426549,-0.995592,1.484856,-0.062689,0.698538,1.613398,-0.012617,-0.163465,0.069105,-0.109660,-0.005854,-0.135878,0.091033,-0.060379,0.636975,-0.225784,1.158276,-0.156289,0.089790,-0.410843,0.025105,0.206257,-0.069730
3,20.0,99.0,95013.0,0.0,1.0,84.0,4.0,1,0,0,0,0,0,101.0,1.0,0,0,0,0,0,0,1,1,0,0,28.0,28.0,28.000000,41.0,2.0,72.0,2.0,72.0,2.0,72.0,194.0,194.0,1.0,1075.00,22102.0,22102.0,674.0,448.0,1.0,43552.0,43552.0,194.0,194.0,1.0,1254.0,918.0,1.0,43552.0,43552.0,1.0,43551.0,43551.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-3.268014,0.686026,2.073008,2.739523,-1.700498,0.795903,-0.366111,-0.171108,-0.255920,-0.076859,0.035128,-0.078693,-0.239493,-0.196342,0.373617,0.094255,-1.108501,0.839687,0.148620,-0.359842,1.396223,0.162028,-0.895299,0.183419,0.767929,0.566074
4,30.0,99.0,95013.0,0.0,4.0,60.0,4.0,1,0,0,0,1,0,101.0,1.0,0,0,0,0,1,0,1,1,0,0,28.0,28.0,28.000000,41.0,2.0,72.0,5.0,169.0,6.0,199.0,215.0,215.0,1.0,0.00,22101.0,22101.0,674.0,448.0,1.0,43552.0,43552.0,215.0,215.0,1.0,1254.0,918.0,1.0,43552.0,43552.0,1.0,43551.0,43551.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-1.854126,0.087949,-0.283453,0.666777,-0.477386,0.052608,0.228457,0.058800,0.124905,0.071982,0.146235,-0.315314,-0.382139,0.062511,0.150419,-0.004474,-0.338486,0.327677,0.166384,-0.517259,0.901367,0.033234,0.236080,-0.114170,0.106021,0.047355
5,35.0,99.0,95013.0,0.0,95.0,50.0,4.0,0,1,1,0,0,0,101.0,1.0,0,0,0,0,0,0,1,0,1,0,28.0,28.0,28.181818,41.0,2.0,75.0,4.0,164.0,6.0,348.0,572.0,479.0,1.0,282011.00,22101.0,22101.0,1254.0,1254.0,1.0,43552.0,43552.0,1254.0,1254.0,1.0,1254.0,642.0,2.0,43553.0,43553.0,2.0,43553.0,43553.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,75,250,334,348,16,45,1.639688,-0.702730,-0.578840,0.403708,-1.169567,1.884454,0.265900,0.869516,1.878994,-0.000306,-0.19

In [28]:
merged_df = pd.concat([df2, df_final], axis=1)

In [30]:
merged_df.shape

(1285402, 69)

In [31]:
merged_df.head()

,CUSTOMER_CODE,batch_date,v24,v44,v45,v51,v52,v53,v54,v63,v81,v82,v87,v88,v92,v95,v99,v103,v104,v105,v106,v173,v175,v271,v273,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v290,v291,v295,v298,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,2024-09-27,0.0,4.0,1,1,1,0,0,0,0.0,0,1,1,1,1,0,1,0,0,0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,103,278,362,21,44,73,-0.322119,-0.918599,-0.273532,-0.559537,-0.386782,-0.089112,1.685201,-0.336442,-1.153682,-0.526113,-0.063434,0.314679,0.202600,0.345686,-0.211092,-0.154966,0.062258,-0.355990,-0.011503,-0.630801,0.055311,-0.097643,0.252998,0.167742,-0.113037,-0.181209
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,2024-09-27,0.0,4.0,1,0,0,1,0,1,0.0,1,0,0,0,0,1,1,1,0,1,2.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,103,278,362,21,44,73,2.553447,0.951081,0.300505,0.231803,-0.014947,-0.501481,-0.235533,0.353483,-0.696045,0.090170,0.981325,1.183267,0.818103,-0.558489,-0.615532,-0.256515,-0.087198,0.350292,-0.971604,-0.149342,0.140371,0.823861,0.101140,0.074615,-0.499478,-0.387337
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,2024-09-27,0.0,4.0,1,1,1,0,0,0,1.0,0,0,1,0,0,0,1,0,1,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,1.056558,-0.682596,-0.784958,0.426549,-0.995592,1.484856,-0.062689,0.698538,1.613398,-0.012617,-0.163465,0.069105,-0.109660,-0.005854,-0.135878,0.091033,-0.060379,0.636975,-0.225784,1.158276,-0.156289,0.089790,-0.410843,0.025105,0.206257,-0.069730
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,2024-09-27,0.0,4.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-3.268014,0.686026,2.073008,2.739523,-1.700498,0.795903,-0.366111,-0.171108,-0.255920,-0.076859,0.035128,-0.078693,-0.239493,-0.196342,0.373617,0.094255,-1.108501,0.839687,0.148620,-0.359842,1.396223,0.162028,-0.895299,0.183419,0.767929,0.566074
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,2024-09-27,0.0,4.0,1,0,0,0,1,0,1.0,0,0,0,0,1,0,1,1,0,0,2.0,5.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-1.854126,0.087949,-0.283453,0.666777,-0.477386,0.052608,0.228457,0.058800,0.124905,0.071982,0.146235,-0.315314,-0.382139,0.062511,0.150419,-0.004474,-0.338486,0.327677,0.166384,-0.517259,0.901367,0.033234,0.236080,-0.114170,0.106021,0.047355


In [32]:
merged_df['batch_date'] = pd.to_datetime(merged_df['batch_date'])

# Extracting the month from the 'batch_date' column
merged_df['batch_month'] = merged_df['batch_date'].dt.month


In [33]:
df3 = pd.read_csv('/kaggle/input/12345678/train_action_history.csv')

In [34]:
df3.head()

,customer_code,Offer_id,Offer_subid,batch_id,product_category,product_sub_category,send_timestamp,open_timestamp
0,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,AC_100044882,VR_200098111,70000078,CC_ACQ_SECURED,SECURED_ACQ,2024-10-22T17:35:27.000Z,NaN
1,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,AC_100048426,VR_200108485,10000253,SIP / MF,ACQUISITION,2024-08-09T18:20:18.000Z,NaN
2,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,AC_100052603,VR_200127708,70000074,CC_INORGANIC,EMI,2024-09-20T10:45:20.000Z,2024-09-20T10:45:31.000Z
3,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,AC_100047006,VR_200105035,70000072,CC_ACQ_SECURED,SECURED_ACQ,2024-09-06T12:37:16.000Z,NaN
4,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,AC_100046122,VR_200100845,10000246,RURAL,BALANCE BUILD UP,2024-06-23T11:03:13.000Z,NaN


In [35]:
import pandas as pd
import numpy as np





# Ensure timestamps are converted to pandas datetime objects
if 'send_timestamp' in df3.columns:
    df3['send_timestamp'] = pd.to_datetime(df3['send_timestamp'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')
if 'open_timestamp' in df3.columns:
    df3['open_timestamp'] = pd.to_datetime(df3['open_timestamp'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')

# Define a function to calculate slots based on vectorized pandas operations
def calculate_slots(timestamps):
    """
    Vectorized function to calculate slots (1-28) for given timestamps.
    """
    # Weekday: Monday=0, Sunday=6
    day_of_week = timestamps.dt.weekday
    
    # Hour: Extract hour from timestamp
    hour = timestamps.dt.hour
    
    # Map hours to slots within a day
    slot_within_day = np.select(
        condlist=[
            (hour >= 9) & (hour < 12),
            (hour >= 12) & (hour < 15),
            (hour >= 15) & (hour < 18),
            (hour >= 18) & (hour < 21),
        ],
        choicelist=[1, 2, 3, 4],
        default=np.nan  # Default is NaN for out-of-range hours
    )
    
    # Calculate the overall slot (1–28)
    overall_slot = (day_of_week * 4 + slot_within_day).astype('Int64')  # Convert to integer
    return overall_slot

# Calculate slots for 'send_timestamp' and 'open_timestamp'
if 'send_timestamp' in df3.columns:
    df3['send_slot'] = calculate_slots(df3['send_timestamp'])

if 'open_timestamp' in df3.columns:
    df3['open_slot'] = calculate_slots(df3['open_timestamp'])

# Display the updated DataFrame
(df3.head(10))


,customer_code,Offer_id,Offer_subid,batch_id,product_category,product_sub_category,send_timestamp,open_timestamp,send_slot,open_slot
0,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,AC_100044882,VR_200098111,70000078,CC_ACQ_SECURED,SECURED_ACQ,2024-10-22 17:35:27,NaT,7,<NA>
1,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,AC_100048426,VR_200108485,10000253,SIP / MF,ACQUISITION,2024-08-09 18:20:18,NaT,20,<NA>
2,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,AC_100052603,VR_200127708,70000074,CC_INORGANIC,EMI,2024-09-20 10:45:20,2024-09-20 10:45:31,17,17
3,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,AC_100047006,VR_200105035,70000072,CC_ACQ_SECURED,SECURED_ACQ,2024-09-06 12:37:16,NaT,18,<NA>
4,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,AC_100046122,VR_200100845,10000246,RURAL,BALANCE BUILD UP,2024-06-23 11:03:13,NaT,25,<NA>
5,006bd909c3df1b3ff63f93430f38477e17575fd2319cab...,AC_100022721,VR_200049971,10000249,SIP / MF,ACQUISITION,2024-07-15 09:50:14,NaT,1,<NA>
6,00866b1506283114cbaec5c578ea48f9eecb79d288316f...,AC_100054482,VR_200139494,50082467,DEPOSITS,FD ACQUISITION,2024-11-28 19:23:24,NaT,16,<NA>
7,008decd99516de88ad41fb9f374dfe7c81330a186fcc96...,AC_100044882,VR_200098111,70000063,CC_ACQ_SECURED,SECURED_ACQ,2024-07-13 18:53:21,NaT,24,<NA>
8,008ff7e2176d5027d9bca73ed0d08d98ebc676fe914e5f...,AC_100032341,VR_200074702,50072760,PL X-SELL,ACEPL,2024-07-02 11:40:17,NaT,5,<NA>
9,0097defa9cceab43ea010002f4780d778e782ed3b77817...,AC_100044882,VR_200098111,70000076,CC_ACQ_SECURED,SECURED_ACQ,2024-10-03 16:03:30,NaT,15,<NA>


In [36]:
# Convert send_timestamp to datetime and extract the month
df3['send_month'] = pd.to_datetime(df3['send_timestamp']).dt.month

df3.head()

,customer_code,Offer_id,Offer_subid,batch_id,product_category,product_sub_category,send_timestamp,open_timestamp,send_slot,open_slot,send_month
0,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,AC_100044882,VR_200098111,70000078,CC_ACQ_SECURED,SECURED_ACQ,2024-10-22 17:35:27,NaT,7,<NA>,10
1,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,AC_100048426,VR_200108485,10000253,SIP / MF,ACQUISITION,2024-08-09 18:20:18,NaT,20,<NA>,8
2,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,AC_100052603,VR_200127708,70000074,CC_INORGANIC,EMI,2024-09-20 10:45:20,2024-09-20 10:45:31,17,17,9
3,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,AC_100047006,VR_200105035,70000072,CC_ACQ_SECURED,SECURED_ACQ,2024-09-06 12:37:16,NaT,18,<NA>,9
4,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,AC_100046122,VR_200100845,10000246,RURAL,BALANCE BUILD UP,2024-06-23 11:03:13,NaT,25,<NA>,6


In [37]:
df3.shape

(8797911, 11)

In [38]:
merged_df.shape

(1285402, 70)

In [39]:
df3.head(10)

,customer_code,Offer_id,Offer_subid,batch_id,product_category,product_sub_category,send_timestamp,open_timestamp,send_slot,open_slot,send_month
0,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,AC_100044882,VR_200098111,70000078,CC_ACQ_SECURED,SECURED_ACQ,2024-10-22 17:35:27,NaT,7,<NA>,10
1,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,AC_100048426,VR_200108485,10000253,SIP / MF,ACQUISITION,2024-08-09 18:20:18,NaT,20,<NA>,8
2,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,AC_100052603,VR_200127708,70000074,CC_INORGANIC,EMI,2024-09-20 10:45:20,2024-09-20 10:45:31,17,17,9
3,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,AC_100047006,VR_200105035,70000072,CC_ACQ_SECURED,SECURED_ACQ,2024-09-06 12:37:16,NaT,18,<NA>,9
4,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,AC_100046122,VR_200100845,10000246,RURAL,BALANCE BUILD UP,2024-06-23 11:03:13,NaT,25,<NA>,6
5,006bd909c3df1b3ff63f93430f38477e17575fd2319cab...,AC_100022721,VR_200049971,10000249,SIP / MF,ACQUISITION,2024-07-15 09:50:14,NaT,1,<NA>,7
6,00866b1506283114cbaec5c578ea48f9eecb79d288316f...,AC_100054482,VR_200139494,50082467,DEPOSITS,FD ACQUISITION,2024-11-28 19:23:24,NaT,16,<NA>,11
7,008decd99516de88ad41fb9f374dfe7c81330a186fcc96...,AC_100044882,VR_200098111,70000063,CC_ACQ_SECURED,SECURED_ACQ,2024-07-13 18:53:21,NaT,24,<NA>,7
8,008ff7e2176d5027d9bca73ed0d08d98ebc676fe914e5f...,AC_100032341,VR_200074702,50072760,PL X-SELL,ACEPL,2024-07-02 11:40:17,NaT,5,<NA>,7
9,0097defa9cceab43ea010002f4780d778e782ed3b77817...,AC_100044882,VR_200098111,70000076,CC_ACQ_SECURED,SECURED_ACQ,2024-10-03 16:03:30,NaT,15,<NA>,10


In [40]:
merged_df.head(10)

,CUSTOMER_CODE,batch_date,v24,v44,v45,v51,v52,v53,v54,v63,v81,v82,v87,v88,v92,v95,v99,v103,v104,v105,v106,v173,v175,v271,v273,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v290,v291,v295,v298,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,batch_month
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,2024-09-27,0.0,4.0,1,1,1,0,0,0,0.0,0,1,1,1,1,0,1,0,0,0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,103,278,362,21,44,73,-0.322119,-0.918599,-0.273532,-0.559537,-0.386782,-0.089112,1.685201,-0.336442,-1.153682,-0.526113,-0.063434,0.314679,0.202600,0.345686,-0.211092,-0.154966,0.062258,-0.355990,-0.011503,-0.630801,0.055311,-0.097643,0.252998,0.167742,-0.113037,-0.181209,9
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,2024-09-27,0.0,4.0,1,0,0,1,0,1,0.0,1,0,0,0,0,1,1,1,0,1,2.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,103,278,362,21,44,73,2.553447,0.951081,0.300505,0.231803,-0.014947,-0.501481,-0.235533,0.353483,-0.696045,0.090170,0.981325,1.183267,0.818103,-0.558489,-0.615532,-0.256515,-0.087198,0.350292,-0.971604,-0.149342,0.140371,0.823861,0.101140,0.074615,-0.499478,-0.387337,9
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,2024-09-27,0.0,4.0,1,1,1,0,0,0,1.0,0,0,1,0,0,0,1,0,1,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,1.056558,-0.682596,-0.784958,0.426549,-0.995592,1.484856,-0.062689,0.698538,1.613398,-0.012617,-0.163465,0.069105,-0.109660,-0.005854,-0.135878,0.091033,-0.060379,0.636975,-0.225784,1.158276,-0.156289,0.089790,-0.410843,0.025105,0.206257,-0.069730,9
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,2024-09-27,0.0,4.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-3.268014,0.686026,2.073008,2.739523,-1.700498,0.795903,-0.366111,-0.171108,-0.255920,-0.076859,0.035128,-0.078693,-0.239493,-0.196342,0.373617,0.094255,-1.108501,0.839687,0.148620,-0.359842,1.396223,0.162028,-0.895299,0.183419,0.767929,0.566074,9
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,2024-09-27,0.0,4.0,1,0,0,0,1,0,1.0,0,0,0,0,1,0,1,1,0,0,2.0,5.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-1.854126,0.087949,-0.283453,0.666777,-0.477386,0.052608,0.228457,0.058800,0.124905,0.071982,0.146235,-0.315314,-0.382139,0.062511,0.150419,-0.004474,-0.338486,0.327677,0.166384,-0.517259,0.901367,0.033234,0.236080,-0.114170,0.106021,0.047355,9
5,b3804b6f986c427848805716c01cdda570e2e1736d406a...,2024-08-30,0.0,4.0,0,1,1,0,0,0,1.0,0,0,0,0,0,0,1,0,1,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,75,250,334,348,16,45,1.639688,-0.702730,-0.578840,0.403708,-1.169567,1.884454,0.265900,0.869516,1.878994,-0.000306,-0.199255,0.224975,0.086502,0.574047,-0.177020,-0.091742,0.150813,-0.352453,-0.381498,0.509441,-0.160286,-0.057225,-0.300369,0.043756,0.140795,-0.031483,8
6,274beb45e3395a781d4ce73b6bc094ecb8bdfcc7ab80da...,2024-07-26,0.0,4.0,1,1,1,0,1,0,1.0,0,0,1,0,0,0,0,0,1,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,40,215,299,313,347,10,-1.464916,0.453659,-1.559027,0.048928,0.185478,-0.432179,-2.105937,0.340136,0.644351,-0.021903,-0.196395,0.311880,0.300553,0.369789,-0.283540,0.032312,0.054087,0.172771,-0.485982,0.488350,-0.051014,-0.028334,-0.132116,0.223307,0.084774,-0.162260,7
7,9f5cf464846d96900a924c8f79293b7c1bd81f0833b805...,2024-05-31,0.0,4.0,0,1,0,1,0,0,1.0,0,0,1,0,0,0,0,0,1,1,2.0,4.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,350,159,243,257,291,318,-0.575636,0.094538,-1.003972,-0.209198,-0.080778,0.473138,-0.620458,0.274042,0.343319,-0.021974,-0.059787,0.332545,0.275157,-0.106765,-0.285417,0.018481,-0.004597,0.393923,0.128249,0.859807,-0.486994,0.081697,-0.487235,0.045488,0.241885,-0.096370,5
8,f688d48aa879ff818129a5b5594f109cc089b8a55c81e7...,2024-10-25,0.0,1.0,1,0,0,0,0,0,1.0,1,0,1,0,0,0,1,1,0,0,2.0,2.0,1.0,1

In [41]:
result_df = pd.merge(df3, merged_df, left_on=['customer_code', 'send_month'], right_on=['CUSTOMER_CODE', 'batch_month'])

In [42]:
result_df.head(10)

,customer_code,Offer_id,Offer_subid,batch_id,product_category,product_sub_category,send_timestamp,open_timestamp,send_slot,open_slot,send_month,CUSTOMER_CODE,batch_date,v24,v44,v45,v51,v52,v53,v54,v63,v81,v82,v87,v88,v92,v95,v99,v103,v104,v105,v106,v173,v175,v271,v273,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v290,v291,v295,v298,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,batch_month
0,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,AC_100044882,VR_200098111,70000078,CC_ACQ_SECURED,SECURED_ACQ,2024-10-22 17:35:27,NaT,7,<NA>,10,00199d3467a7191db5bfa4e5f9a62eeb96fb0b602c3ec5...,2024-10-25,0.0,2.0,1,0,0,0,1,0,1.0,1,0,1,0,0,1,1,1,0,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,131,306,24,49,72,101,1.486508,-0.777378,-1.842406,-1.196691,0.800100,-0.836045,1.132395,0.090556,-0.508233,0.149978,-0.198671,0.834170,1.416712,0.991704,0.244999,-0.379305,-0.434230,-2.491124,-0.154860,-2.899052,1.171740,-0.383301,-0.175675,0.286302,0.994498,1.187674,10
1,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,AC_100048426,VR_200108485,10000253,SIP / MF,ACQUISITION,2024-08-09 18:20:18,NaT,20,<NA>,8,001f2abab1bccc25d00bba68fea57a81cab1c76d485515...,2024-08-30,0.0,3.0,1,0,0,0,1,1,1.0,1,0,0,0,0,0,1,1,0,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,75,250,334,348,16,45,-1.118049,-2.631199,4.706099,-0.658180,1.880832,0.219725,-1.753331,0.180440,0.992935,0.184490,0.180767,-0.289241,0.070882,-0.712995,0.696783,0.417549,-0.355513,-0.164139,-0.030594,-0.993475,1.067411,-0.039121,-0.387161,0.056855,-0.472688,-1.373892,8
2,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,AC_100052603,VR_200127708,70000074,CC_INORGANIC,EMI,2024-09-20 10:45:20,2024-09-20 10:45:31,17,17,9,00298fc11fb6924004c041f141f92c3c74e209d9a737d9...,2024-09-27,0.0,4.0,1,0,0,1,0,0,1.0,0,0,1,0,1,1,1,0,0,1,2.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,103,278,362,21,44,73,-0.363072,-0.185040,-0.752673,0.164697,-0.151125,-0.055321,0.048758,0.041517,-0.074916,0.002480,-0.091934,-0.048568,0.125988,-0.112059,-0.081630,0.081955,0.183805,0.082610,-0.006269,-0.466157,0.478654,-0.020347,0.507461,-0.069908,-0.207661,-0.195390,9
3,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,AC_100047006,VR_200105035,70000072,CC_ACQ_SECURED,SECURED_ACQ,2024-09-06 12:37:16,NaT,18,<NA>,9,0038a46221c0175fc8938ebc8aef8d0f83b3ac1ad84662...,2024-09-27,0.0,1.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-1.235178,0.082097,-0.291548,0.790197,-0.568126,0.071640,-0.020386,-0.039608,-0.254586,-0.025741,0.419732,-0.298531,-0.755023,0.431661,-0.389833,-0.172576,0.357750,0.324915,-0.114383,0.543712,-0.893824,-0.116075,2.779313,-0.524382,0.034400,-0.112720,9
4,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,AC_100046122,VR_200100845,10000246,RURAL,BALANCE BUILD UP,2024-06-23 11:03:13,NaT,25,<NA>,6,00409395a831af7fd41d0ec70a5be3bda13d64cb3e4be9...,2024-06-28,0.0,3.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,3.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,12,187,271,285,319,346,1.269837,1.352844,-1.179711,-1.371025,0.648154,-3.689853,1.768092,-0.850028,1.476956,0.252468,-0.720980,-2.650092,-0.458265,1.365117,-0.211175,0.118271,-0.072954,-1.218097,0.923972,-0.774798,-0.053545,-0.294380,-0.889092,0.214042,0.432289,0.465100,6
5,006bd909c3df1b3ff63f93430f38477e17575fd2319cab...,AC_100022721,VR_200049971,10000249,SIP / MF,ACQUISITION,2024-07-15 09:50:14,NaT,1,<NA>,7,006bd909c3df1b3ff63f93430f38477e17575fd2319cab...,2024-07-26,0.0,3.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,40,215,299,313,347,10,-1.326692,0.050586,0.279744,1.060402,-0.689880,0.337568,-0.191106,-0.021385,0.029614,-0.040732,-0.388889,-0.158265,0.376600,-0.907587,0.422271,0.414333,-0.328449,0.382029,0.060814,-0.381481,1.089325,0.043545,-0.746771,0.039909,-1.418450

In [43]:
result_df.shape

(7289023, 81)

In [44]:


# List of columns to drop
columns_to_drop = [
    'CUSTOMER_CODE', 'batch_date', 'batch_month', 'send_timestamp', 'open_timestamp', 'send_month', 'Offer_subid', 'batch_id', 'product_category_grouped', 'product_sub_category' 
]

# Drop the specified columns
result_df = result_df.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
result_df.head(10)

In [ ]:
unique_values = result_df['product_category'].unique()
unique_values

In [ ]:
unique_values = result_df['product_sub_category'].unique()
unique_values

In [ ]:
unique_customer_count = result_df['Offer_id'].nunique()
unique_customer_count

In [ ]:
unique_customer_count = result_df['Offer_subid'].nunique()
unique_customer_count

In [ ]:
unique_customer_count = result_df['batch_id'].nunique()
unique_customer_count

In [ ]:
unique_customer_count = result_df['product_category'].nunique()
unique_customer_count

In [ ]:
unique_customer_count = result_df['product_sub_category'].nunique()
unique_customer_count

In [ ]:
print(1)

In [45]:
# Mapping dictionary
category_mapping = {
    'CC_ACQ_SECURED': 'Credit Card Services',
    'CC_ACQ_UNSECURED': 'Credit Card Services',
    'CC_INORGANIC': 'Credit Card Services',
    'CC_UPGRADE': 'Credit Card Services',
    'CC_SERVICE': 'Credit Card Services',
    'CC_SPENDS': 'Credit Card Services',
    'CC_ACTIVATION': 'Credit Card Services',
    'CC_VAS': 'Credit Card Services',
    'CC_REMARKETING': 'Credit Card Services',
    'SIP / MF': 'Investment & Wealth Management',
    'INSURANCE': 'Investment & Wealth Management',
    'NFO': 'Investment & Wealth Management',
    'IPO': 'Investment & Wealth Management',
    'PIS': 'Investment & Wealth Management',
    'LAS': 'Investment & Wealth Management',
    'FX': 'Investment & Wealth Management',
    'SAVINGS': 'Banking Accounts',
    'CURRENT ACCOUNT': 'Banking Accounts',
    'DEPOSITS': 'Banking Accounts',
    'NRI': 'Banking Accounts',
    'ASSETS': 'Loans & Assets',
    'PL X-SELL': 'Loans & Assets',
    'RURAL': 'Loans & Assets',
    'RELATIONSHIP BANKING': 'Relationship & Corporate Banking',
    'CORPSAL': 'Relationship & Corporate Banking',
    'DEBIT CARD': 'Payment & Transaction Services',
    'OPTIMUS': 'Payment & Transaction Services'
}

# Apply the mapping
result_df['product_category'] = result_df['product_category'].map(category_mapping)



In [ ]:
result_df.head()

In [46]:
result_df['Offer_id'] = result_df['Offer_id'].str.replace('AC_1000', '', regex=False).astype(int)

In [47]:
C_code = result_df['customer_code']

In [48]:


# List of columns to drop
columns_to_drop = [
    'customer_code',
]

# Drop the specified columns
result_df = result_df.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
result_df.head()

In [49]:
one_hot_encoded = pd.get_dummies(result_df['product_category'], prefix='Category')

In [50]:
one_hot_encoded.columns = one_hot_encoded.columns.str.replace(' ', '_').str.replace('&', 'and')

In [51]:
result_df = pd.concat([result_df, one_hot_encoded], axis=1)

In [ ]:
result_df.head()

In [52]:


# List of columns to drop
columns_to_drop = [
    'product_category',
]

# Drop the specified columns
result_df = result_df.drop(columns=columns_to_drop, errors='ignore')


In [53]:
result_df['Category_Banking_Accounts'] = result_df['Category_Banking_Accounts'].astype(int)
result_df['Category_Credit_Card_Services'] = result_df['Category_Credit_Card_Services'].astype(int)
result_df['Category_Investment_and_Wealth_Management'] = result_df['Category_Investment_and_Wealth_Management'].astype(int)
result_df['Category_Loans_and_Assets'] = result_df['Category_Loans_and_Assets'].astype(int)
result_df['Category_Payment_and_Transaction_Services'] = result_df['Category_Payment_and_Transaction_Services'].astype(int)
result_df['Category_Relationship_and_Corporate_Banking'] = result_df['Category_Relationship_and_Corporate_Banking'].astype(int)

In [ ]:
result_df.columns

In [54]:
new_column_names = {result_df.columns[-6]: 'c1',
                    result_df.columns[-5]: 'c2',
                    result_df.columns[-4]: 'c3',
                    result_df.columns[-3]: 'c4',
                    result_df.columns[-2]: 'c5',
                    result_df.columns[-1]: 'c6'}

# Rename columns
result_df.rename(columns=new_column_names, inplace=True)

In [55]:
result_df.head()

,Offer_id,send_slot,open_slot,v24,v44,v45,v51,v52,v53,v54,v63,v81,v82,v87,v88,v92,v95,v99,v103,v104,v105,v106,v173,v175,v271,v273,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v290,v291,v295,v298,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,c1,c2,c3,c4,c5,c6
0,44882,7,<NA>,0.0,2.0,1,0,0,0,1,0,1.0,1,0,1,0,0,1,1,1,0,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,131,306,24,49,72,101,1.486508,-0.777378,-1.842406,-1.196691,0.800100,-0.836045,1.132395,0.090556,-0.508233,0.149978,-0.198671,0.834170,1.416712,0.991704,0.244999,-0.379305,-0.434230,-2.491124,-0.154860,-2.899052,1.171740,-0.383301,-0.175675,0.286302,0.994498,1.187674,0,1,0,0,0,0
1,48426,20,<NA>,0.0,3.0,1,0,0,0,1,1,1.0,1,0,0,0,0,0,1,1,0,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,75,250,334,348,16,45,-1.118049,-2.631199,4.706099,-0.658180,1.880832,0.219725,-1.753331,0.180440,0.992935,0.184490,0.180767,-0.289241,0.070882,-0.712995,0.696783,0.417549,-0.355513,-0.164139,-0.030594,-0.993475,1.067411,-0.039121,-0.387161,0.056855,-0.472688,-1.373892,0,0,1,0,0,0
2,52603,17,17,0.0,4.0,1,0,0,1,0,0,1.0,0,0,1,0,1,1,1,0,0,1,2.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,103,278,362,21,44,73,-0.363072,-0.185040,-0.752673,0.164697,-0.151125,-0.055321,0.048758,0.041517,-0.074916,0.002480,-0.091934,-0.048568,0.125988,-0.112059,-0.081630,0.081955,0.183805,0.082610,-0.006269,-0.466157,0.478654,-0.020347,0.507461,-0.069908,-0.207661,-0.195390,0,1,0,0,0,0
3,47006,18,<NA>,0.0,1.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,103,278,362,21,44,73,-1.235178,0.082097,-0.291548,0.790197,-0.568126,0.071640,-0.020386,-0.039608,-0.254586,-0.025741,0.419732,-0.298531,-0.755023,0.431661,-0.389833,-0.172576,0.357750,0.324915,-0.114383,0.543712,-0.893824,-0.116075,2.779313,-0.524382,0.034400,-0.112720,0,1,0,0,0,0
4,46122,25,<NA>,0.0,3.0,1,0,0,0,0,0,1.0,0,0,0,0,0,0,1,1,0,0,3.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,12,187,271,285,319,346,1.269837,1.352844,-1.179711,-1.371025,0.648154,-3.689853,1.768092,-0.850028,1.476956,0.252468,-0.720980,-2.650092,-0.458265,1.365117,-0.211175,0.118271,-0.072954,-1.218097,0.923972,-0.774798,-0.053545,-0.294380,-0.889092,0.214042,0.432289,0.465100,0,0,0,1,0,0


In [56]:
result_df.shape

(7289023, 76)

In [ ]:
result_df.to_csv('FinallyDone.csv', index = False)

KeyboardInterrupt: 

In [ ]:
print(1)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/working/DataPrepDone.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df_opened = df[df['open_slot'].notna()]
df_closed = df[df['open_slot'].isna()] 

In [ ]:
df_opened.shape

In [ ]:
df_closed.shape

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer